# Bert Model Registration

This Jupyter Notebook implements a BERT-based similarity model using MLflow for tracking, managing, and deploying the model. It loads a pre-trained BERT model, computes sentence embeddings, and retrieves the most similar sentences from a stored corpus based on cosine similarity.

# Notebook Overview
- Import dependencies
- Configure Logging and Define Constants and Paths
- Downloading the Bert Large Uncased Model
- Defining the BERT Tourism Model Class
- Logging Model to MLflow
- Fetching the Latest Model Version from MLflow
- Loading the Model and Running Inference
- Displaying Results for the Input Query

# Import Dependencies

In [1]:
# Standard Library Imports
import os
import json
import shutil
import logging

# Third-Party Libraries
import torch
import numpy as np
import pandas as pd
from tabulate import tabulate
from sklearn.metrics.pairwise import cosine_similarity

# MLflow for Experiment Tracking and Model Management
import mlflow
import mlflow.pyfunc
from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, TensorSpec, ParamSchema, ParamSpec

# Transformers and NLP Libraries
from transformers import AutoTokenizer
from nemo.collections.nlp.models.language_modeling import BERTLMModel

## Configure Logging and Define Constants and Paths

In [2]:
# Create logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S") 

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

CORPUS_PATH = "../data/raw/corpus.csv"
EMBEDDINGS_PATH = "../data/processed/embeddings.csv"
BERT_MODEL_NAME = "bert-large-uncased"
BERT_MODEL_ONLINE_PATH = "/root/.cache/torch/NeMo/NeMo_1.22.0/bertlargeuncased/ca4ebba9f05a8ffb79845249ca046983/bertlargeuncased.nemo"
BERT_MODEL_DATAFABRIC_PATH = "/home/jovyan/datafabric/Bertlargeuncased/bertlargeuncased.nemo"
DEMO_PATH = "../demo"
EXPERIMENT_NAME = "BERT_Tourism_Experiment"
RUN_NAME = "BERT_Tourism_Run"
MODEL_NAME = "BERT_Tourism_Model"

In [3]:
logger.info('Notebook execution started.')

2025-04-02 20:31:38 - INFO - Notebook execution started.


# Downloading the Bert Large Uncased Model

In [4]:
# Ensure you have added the 'bertlargeuncased' model from the NVIDIA NGC model catalog.
# If unavailable, uncomment the following line and use the alternative method below to download the BERT model online.
# bert_model = BERTLMModel.from_pretrained(model_name="bertlargeuncased", strict=False).to(device)

# Defining the BERT Tourism Model Class

In [5]:
class BERTTourismModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        Load precomputed embeddings, corpus, and the pre-trained BERT model.
        """
        # Load precomputed embeddings and corpus data
        self.embeddings_df = pd.read_csv(context.artifacts['embeddings_path'])
        self.corpus_df = pd.read_csv(context.artifacts['corpus_path'])
        
        # Load tokenizer for BERT
        self.tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
        
        # Set device to GPU if available, otherwise use CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Load pre-trained BERT model
        self.bert_model = BERTLMModel.restore_from(context.artifacts['bert_model_path'], strict=False).to(self.device)
    
    def generate_query_embedding(self, query):
        """
        Generate BERT embeddings for the input query.
        """
        self.bert_model.eval()  # Set model to evaluation mode
        
        # Tokenize the input query and move tensors to the selected device
        encoded_input = self.tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=128)
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}
        
        # Get the model's output embedding
        with torch.no_grad():
            output = self.bert_model.bert_model(**encoded_input)
        
        # Return the [CLS] token embedding as a NumPy array
        return output[:, 0, :].cpu().numpy()
    
    def predict(self, context, model_input, params):
        """
        Compute similarity between query and precomputed embeddings,
        then return the top 5 most similar results.
        """
        # Extract the query string from model input
        query = model_input["query"][0]
        
        # Generate query embedding
        query_embedding = self.generate_query_embedding(query)
        
        # Compute cosine similarity between query and precomputed embeddings
        similarities = cosine_similarity(query_embedding, self.embeddings_df.values)
        
        # Get indices of top 5 most similar results
        top_indices = np.argsort(similarities[0])[::-1][:5]
        
        # Retrieve corresponding results from the corpus
        results = self.corpus_df.iloc[top_indices].copy()
        results.loc[:, 'Similarity'] = similarities[0][top_indices]
        
        # Return results as a dictionary
        return results.to_dict(orient="records")
    
    @classmethod
    def log_model(cls, model_name):
        """
        Logs the model to MLflow with appropriate artifacts and schema.
        """
        # Define input and output schema
        input_schema = Schema([ColSpec("string", "query")])
        output_schema = Schema([
            TensorSpec(np.dtype("object"), (-1,), "List of Pledges and Similarities")
        ])
        params_schema = ParamSchema([ParamSpec("show_score", "boolean", False)])
        
        # Define model signature
        signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=params_schema)
        
        # Log the model in MLflow
        mlflow.pyfunc.log_model(
            model_name,
            python_model=cls(),
            artifacts={
                "corpus_path": CORPUS_PATH,
                "embeddings_path": EMBEDDINGS_PATH, 
                # If you are using the downloaded bert model then uncomment the line below and comment the other bert model line that uses nemo model from datafabric
                #"bert_model_path": BERT_MODEL_ONLINE_PATH,            
                "bert_model_path": BERT_MODEL_DATAFABRIC_PATH,
                "demo": DEMO_PATH,
            },
            signature=signature
        )

 # Logging Model to MLflow

In [6]:
logger.info(f'Starting the experiment: {EXPERIMENT_NAME}')

# Set the MLflow experiment name
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

# Start an MLflow run
with mlflow.start_run(run_name=RUN_NAME) as run:
    # Print the artifact URI for reference
    logging.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    
    # Log the BERT similarity model to MLflow
    BERTTourismModel.log_model(model_name=MODEL_NAME)

    # Register the logged model in MLflow Model Registry
    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/{MODEL_NAME}", 
        name=MODEL_NAME
    )

logger.info(f'Registered the model: {MODEL_NAME}')

2025-04-02 20:31:38 - INFO - Starting the experiment: BERT_Tourism_Experiment


2025/04/02 20:32:20 WARNING mlflow.utils.requirements_utils: Found transformer-engine version (0.13.0+8eae4ce) contains a local version label (+8eae4ce). MLflow logged a pip requirement for this package as 'transformer-engine==0.13.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Registered model 'BERT_Tourism_Model' already exists. Creating a new version of this model...
Created version '8' of model 'BERT_Tourism_Model'.
2025-04-02 20:32:43 - INFO - Registered the model: BERT_Tourism_Model


# Fetching the Latest Model Version from MLflow

In [7]:
# Initialize the MLflow client
client = MlflowClient()

# Retrieve the latest version of the "BERT_Tourism_Model" model (not yet in a specific stage)
model_metadata = client.get_latest_versions(MODEL_NAME, stages=["None"])
latest_model_version = model_metadata[0].version  # Extract the latest model version

# Fetch model information, including its signature
model_info = mlflow.models.get_model_info(f"models:/{MODEL_NAME}/{latest_model_version}")

# Print the latest model version and its signature
print(f"Latest Model Version: {latest_model_version}")
print(f"Model Signature: {model_info.signature}")

[NeMo W 2025-04-02 20:32:43 nemo_logging:349] /tmp/ipykernel_1504/758826530.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
      model_metadata = client.get_latest_versions(MODEL_NAME, stages=["None"])
    


Latest Model Version: 8
Model Signature: inputs: 
  ['query': string (required)]
outputs: 
  ['List of Pledges and Similarities': Tensor('object', (-1,))]
params: 
  ['show_score': boolean (default: False)]



# Loading the Model and Running Inference

In [8]:
# Load the trained BERT similarity model from MLflow
model = mlflow.pyfunc.load_model(model_uri=f"models:/{MODEL_NAME}/{latest_model_version}")

# Define a sample query for testing
query = "Give me a resort budget vacation suggestion"

# Use the model to predict similar results based on the query
result = model.predict({"query": [query]})

[NeMo W 2025-04-02 20:34:07 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    data_file: /home/yzhang/data/nlp/bert/47316/hdf5/lower_case_1_seq_len_512_max_pred_80_masked_lm_prob_0.15_random_seed_12345_dupe_factor_5_shard_1472_test_split_10/books_wiki_en_corpus/training/
    max_predictions_per_seq: 80
    batch_size: 16
    shuffle: true
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
    
[NeMo W 2025-04-02 20:34:09 modelPT:617] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2025-04-02 20:34:09 modelPT:728] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 4.375e-05
        maximize: False
        weight_decay: 0.01
    )


[NeMo W 2025-04-02 20:34:09 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2025-04-02 20:34:11 save_restore_connector:249] Model BERTLMModel was successfully restored from /phoenix/mlflow/767574194519062736/3eee25f0e4c34f2ab404131cfb338dd0/artifacts/BERT_Tourism_Model/artifacts/bertlargeuncased.nemo.


# Displaying Results for the Input Query

In [9]:
# Convert the result into a pandas DataFrame
df = pd.DataFrame(result)

# Drop unnecessary columns if needed
df = df.drop(columns=["Unnamed: 0", "Topic"], errors="ignore")

# Rename columns for better readability
df.rename(columns={"Pledge": "Recommended Option", "Similarity": "Relevance Score"}, inplace=True)

# Display the DataFrame in a tabular format
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

╒════╤═════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════╕
│    │ Recommended Option                                                                                  │   Relevance Score │
╞════╪═════════════════════════════════════════════════════════════════════════════════════════════════════╪═══════════════════╡
│  0 │ For a budget-friendly vacation, consider a resort with vacation options and cruise activities.      │          0.869171 │
├────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│  1 │ For a budget-friendly vacation, consider a getaway with beach options and vacation activities.      │          0.863892 │
├────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│  2 │ For a budget-friendly vacation, consider a getaway with hotel options and vacation activit

In [10]:
logger.info('Notebook execution completed.')

2025-04-02 20:34:11 - INFO - Notebook execution completed.
